## Imports

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm

## Configuration

In [4]:
# Set the plotting style to seaborn instead of matplotlib
sns.set_theme()

# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

# Free up cuda cache on reruns
if device == 'cuda':
    torch.cuda.empty_cache()

Using device: cuda


## Preprocessing Util

## Data Loading

## Model Design

In [ ]:

class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvLayer, self).__init__()
        self.block = nn.Sequential([nn.Conv2d(in_channels, out_channels, kernel_size=3),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(2)])

class ConvBlock(nn.Module):
    def __init__(self, in_channels=3):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential([ConvLayer(in_channels, 32),
                                    ConvLayer(32, 64),
                                    ConvLayer(64, 128),
                                    ConvLayer(128, 256),
                                    nn.Flatten()])
    
    def forward(self, x: torch.Tensor):
        return self.block(x)
    
class TimeDistributed(nn.Module):
    def __init__(self, module, sequence_length, *args, **kwargs):
        super(TimeDistributed, self).__init__()
        self.modules = nn.ModuleList([module(*args, **kwargs) for _ in range(sequence_length)])
        
    def forward(self, x: torch.Tensor):
        # x is of shape (batch_size, sequence_length, channels, height, width)
        return torch.stack([layer(x[:,i,:,:,:].squeeze()) for i, layer in enumerate(self.modules)], dim=1)
    
    
    
class RecurrentBlock(nn.Module):
    def __init__(self, sequence_length):
        super(RecurrentBlock, self).__init__()
        self.hidden = None
        self.lstm = nn.LSTM(256, 128, bidirectional=True, batch_first=True, num_layers=sequence_length)
    
    def forward(self, x):
        # x is of shape (batch_size, sequence_length, features)
        output, self.hidden = self.lstm(x, self.hidden)
        return output
    

class LinearBlock(nn.Module):
    def __init__(self):
        super(LinearBlock, self).__init__()
        self.block = nn.Sequential([nn.Linear(128, 64),
                                   nn.ReLU(),
                                   nn.Dropout(0.5),
                                   nn.Linear(64, 5),
                                   nn.Sigmoid()])
        
    def forward(self, x: torch.Tensor):
        # x is of shape (batch_size, features)
        return self.block(x)
    

class FearNet(nn.Module):
    def __init__(self, sequence_length=10):
        super(FearNet, self).__init__()
        self.tcnn = TimeDistributed(ConvBlock, sequence_length=sequence_length)
        self.rnn = RecurrentBlock(sequence_length=sequence_length)
        self.tail = LinearBlock()
    
    def forward(self, x: torch.Tensor):
        # x is of shape (batch_size, sequence_length, channels, height, width)
        x = self.tcnn(x)
        # x is of shape (batch_size, sequence_length, features)
        self.rnn(x)
        # Getting the last hidden state from the last layer
        x = self.rnn.hidden[0][-1]
        # x is of shape (batch_size, features)
        x = self.tail(x)
        return x
        
